# Chapter9
In this chapter we look at:
* Classes and class relationships
* Syntax/code-idioms
* Exhaustive search
* Previous Chapters

In [chapter 1](chapter1.ipynb) (a closely related notebook see hyperlink) We look at optimization outputs that came from an optimization using spike waveform shape. 

In [chapter 2](chapter2.ipynb) (a closely related notebook see hyperlink) I demonstrate optimization using spike time statistics via the allen SDK Chapter 2

In [chapter 3](chapter3.ipynb) (a closely related notebook see hyperlink) We will take a closer at the neuroelectro data used to perform the fits in notebook1/Chapter 1. Specifically we will look at where optimized model behavior fits back onto the distribution of neuroelectro data.

In [chapter 5](chapter5.ipynb) (a closely related notebook see hyperlink) We will look at projections of Optimized cells onto a Druckman feature space, we will also look at extracting Allen SDK features from the optimized cells. Chapter 3


# Syntax and Idioms
## Data Transport Container
The main way of storing model attributes and moving non-parallel models tests around on different CPUs

Class Definition is in /neuronunit/neuronunit/optimisation/data_transport_container.py

Inputs and outputs of Optimization, are typically Data Transport Containers. 

dtc objects now contain a method:

dtc.dtc_to_model()

## Test Suite Dictionary
An extension of pythons dictionary class. It is capable of calling a method ```optimize``` on itself.

* This type also has an attribute, use_rheobase in scores.

* Since not all experimental cell test suites, use rheobase to judge model fitness (Allen SDK does not). It's simpler to decide if rheobase calculations will contribute to Neuronunit scores at the users entrypoint to optimization.

 /neuronunit/neuronunit/optimisation/optimization_management.py

## Class OptMan
* An Optimization Management Object
Located at
 /neuronunit/neuronunit/optimisation/optimization_management.py

This object contains the method ```run_ga```, ```round_trip_test``` and ```run_grid```.
The object contains logic for switching between elephant and allen based measurements of waveform and firing. This object also contains logic for rheobase guided searches, or many firing waveform searches.

Once an optimization has been described using the Optmization management object a secondary object is constructed.

The ```OptMan``` object also performs tasks like creating new genes to replace models that had unstable parameters

Examples of using the OptMan class to optimize:


``` python
OM = OptMan(local_tests,backend=str('RAW'),boundary_dict=MODEL_PARAMS['RAW'],protocol={'elephant':True,'allen':False,'dm':False})#'tsr':spk_range})

grid_results = OM.run_simple_grid(5,free_params=['a','b','c'])
```

To simulate data, and internall check multi-objective high dimensional optimization:

``` python

OM = OptMan(use_test,protocol={'elephant':True,'allen':False,'dm':False})
results,converged,target,simulated_tests = OM.round_trip_test(use_test,
                                                              str('RAW'),\
                                                              MU=10,NGEN=10,stds = easy_standards)

```
To optimize against external neuro-electro data with an axaptive exponential backend:
``` python
ga_outad, DO = OM.run_ga(model_params.MODEL_PARAMS['ADEXP'],3, local_tests, free_params = model_params.MODEL_PARAMS['ADEXP'], MU = 10, backend = str('ADEXP'))
```
The run_ga method is now wrapped inside a ***Neuronunit Test Suite*** dictionary class. 

``` python
# test_dic an actual simple dictionary of NU Tests:
keys: test names, values test objects:

sim_tests = TSD(test_dic)

dtcpop = sim_tests.optimize(model_parameters.MODEL_PARAMS[backend], NGEN=50, \
                             backend=backend, MU=24, protocol={'allen': True, 'elephant': False,'tsr':results['dtc_pop'][0].tsr})

```

This Object constructs another object called  ```SciUnitOptimization``` that takes care of deap configuration. This file is departs from BluePyOpts base class, but it is similar to it.

The class sciunitoptimization largely calls methods in the files:
* ***/neuronunit/optimisation/algorithms.py***
* ***/neuronunit/optimisation/optimisations.py***

## These two files still largely follow BluePyOpt and DEAP conventions (expressed in DEAPS nsga2.py example file):

* ***/neuronunit/optimisation/algorithms.py***
* ***/neuronunit/optimisation/optimisations.py***

* With the differences: dask.bag.map is used to parallel map instead of scoop etc.

* nsga3 is used in the place of nsga2

* algorithms -> optimization_management.py ```def evaluate()``` 


This class (appropriated from BluePyOpt) is used to create new genes to replace models with unstable behavior.
```python
class WSListIndividual(list):
    """Individual consisting of list with weighted sum field"""
    def __init__(self, *args, **kwargs):
        """Constructor"""
        self.rheobase = None
        super(WSListIndividual, self).__init__(*args, **kwargs)

```



In a very simple example below. The OptMan class is instanced and used to perform simple optimization jobs including:
Exhaustive Search, running a genetic algorithm (nsga3), or round trip testing with simulated data


In [1]:
%%capture
from allensdk.ephys.extract_cell_features import extract_cell_features
import pickle
from neuronunit.optimisation.optimization_management import OptMan, TSD
from neuronunit.optimisation import model_parameters
from neuronunit.optimisation.model_parameters import MODEL_PARAMS
import pandas as pd
import os



In [ ]:

electro_path = str(os.getcwd())+'/../tests/russell_tests.p'

assert os.path.isfile(electro_path) == True
with open(electro_path,'rb') as f:
    (test_frame,obs_frame) = pickle.load(f)

In [ ]:
#filtered_tests = {key:val for key,val in test_frame.items()}
#spk_range = [spk-6,spk+6]
local_tests = test_frame['Hippocampus CA1 pyramidal cell']
local_tests = TSD(local_tests)
local_tests.use_rheobase_score = True
OM = OptMan(local_tests,backend=str('RAW'),boundary_dict=MODEL_PARAMS['RAW'],protocol={'elephant':True,'allen':False,'dm':False})#'tsr':spk_range})
# restrict the number of free parameters to 3, to make the grid search computationally tractable.
grid_results = OM.run_simple_grid(5,free_params=['a','b','c'])
[ (gr.dtc.scores,gr.dtc.attrs) for gr in grid_results ] 

In [7]:
[ (gr.dtc.scores,gr.dtc.attrs) for gr in grid_results ] 

[({'RheobaseTest': 8.47477458554291e-05,
   'RestingPotentialTest': 0.36816528919597813,
   'InjectedCurrentAPThresholdTest': 0.9797604836255633,
   'TimeConstantTest': 0.7028267435086151,
   'InjectedCurrentAPAmplitudeTest': 0.9980353270758306,
   'CapacitanceTest': 0.265108402667218,
   'InjectedCurrentAPWidthTest': 0.47951519668537346,
   'InputResistanceTest': 1.0},
  {'a': 0.01, 'b': -2.0, 'c': -60.0}),
 ({'RheobaseTest': 8.47477458554291e-05,
   'RestingPotentialTest': 0.36816528919597813,
   'InjectedCurrentAPThresholdTest': 0.9797604836255633,
   'TimeConstantTest': 0.7028267435086151,
   'InjectedCurrentAPAmplitudeTest': 0.9980353270758306,
   'CapacitanceTest': 0.265108402667218,
   'InjectedCurrentAPWidthTest': 0.47951519668537346,
   'InputResistanceTest': 1.0},
  {'a': 0.01, 'b': -2.0, 'c': -55.0}),
 ({'RheobaseTest': 8.47477458554291e-05,
   'RestingPotentialTest': 0.36816528919597813,
   'InjectedCurrentAPThresholdTest': 0.9797604511070626,
   'TimeConstantTest': 0.70282

[]

True
Z = 0.48
{'value': array(-79.) * mV, 'mean': array(-79.) * mV, 'n': 2578, 'std': array(28.74658531) * mV} {'mean': array(-0.06522619) * V, 'std': array(1.) * V}
Z = 0.48
{'value': array(-79.) * mV, 'mean': array(-79.) * mV, 'n': 2578, 'std': array(28.74658531) * mV} {'mean': array(-0.06522619) * V, 'std': array(-0.9783928) * V}
True
True
True
True
True
{'amplitude': array(0.07870169) * pA, 'delay': array(100.) * ms, 'duration': array(1000.) * ms}
{'amplitude': array(0.07870169) * pA, 'delay': array(100.) * ms, 'duration': array(1000.) * ms}
{'amplitude': array(0.07870169) * pA, 'delay': array(100.) * ms, 'duration': array(1000.) * ms}
{'amplitude': array(0.07870169) * pA, 'delay': array(100.) * ms, 'duration': array(1000.) * ms}
True
True


{'RheobaseTest': 8.47477458554291e-05,
 'RestingPotentialTest': 0.36816528919597813,
 'InjectedCurrentAPThresholdTest': 0.9797604836255633,
 'TimeConstantTest': 0.7028267435086151,
 'InjectedCurrentAPAmplitudeTest': 0.9980353270758306,
 'CapacitanceTest': 0.265108402667218,
 'InjectedCurrentAPWidthTest': 0.47951519668537346,
 'InputResistanceTest': 1.0}

In [6]:
[ (gr.dtc.scores,gr.dtc.attrs) for gr in grid_results ] 

[({'RheobaseTest': 8.47477458554291e-05,
   'RestingPotentialTest': 0.36816528919597813,
   'InjectedCurrentAPThresholdTest': 0.9797604836255633,
   'TimeConstantTest': 0.7028267435086151,
   'InjectedCurrentAPAmplitudeTest': 0.9980353270758306,
   'CapacitanceTest': 0.265108402667218,
   'InjectedCurrentAPWidthTest': 0.47951519668537346,
   'InputResistanceTest': 1.0},
  {'a': 0.01, 'b': -2.0, 'c': -60.0}),
 ({'RheobaseTest': 8.47477458554291e-05,
   'RestingPotentialTest': 0.36816528919597813,
   'InjectedCurrentAPThresholdTest': 0.9797604836255633,
   'TimeConstantTest': 0.7028267435086151,
   'InjectedCurrentAPAmplitudeTest': 0.9980353270758306,
   'CapacitanceTest': 0.265108402667218,
   'InjectedCurrentAPWidthTest': 0.47951519668537346,
   'InputResistanceTest': 1.0},
  {'a': 0.01, 'b': -2.0, 'c': -55.0}),
 ({'RheobaseTest': 8.47477458554291e-05,
   'RestingPotentialTest': 0.36816528919597813,
   'InjectedCurrentAPThresholdTest': 0.9797604511070626,
   'TimeConstantTest': 0.70282

# Optimization management classes 
can also do things like round-trip testing (test the simulators internal performance, as if to bench mark itself with simulated neural data).

In [8]:
res = OM.round_trip_test(local_tests,str('RAW'),MU=1,NGEN=1)#,stds = easy_standards)


{'amplitude': array(0.21953301) * pA, 'delay': array(100.) * ms, 'duration': array(1000.) * ms}
{'amplitude': array(0.21953301) * pA, 'delay': array(100.) * ms, 'duration': array(1000.) * ms}
True
True
Z = inf
{'mean': array(-0.06844031) * V, 'std': array(0.) * V} {'mean': array(-0.06522619) * V, 'std': array(1.) * V}
Z = inf
{'mean': array(-0.06844031) * V, 'std': array(0.) * V} {'mean': array(-0.06522619) * V, 'std': array(-0.9783928) * V}
True
True
True
True
True
{'amplitude': array(0.0785633) * pA, 'delay': array(100.) * ms, 'duration': array(1000.) * ms}
{'amplitude': array(0.0785633) * pA, 'delay': array(100.) * ms, 'duration': array(1000.) * ms}
{'amplitude': array(0.0785633) * pA, 'delay': array(100.) * ms, 'duration': array(1000.) * ms}
{'amplitude': array(0.0785633) * pA, 'delay': array(100.) * ms, 'duration': array(1000.) * ms}
True
True
1
[[50, 0.01, -2, -60, 10]]
True
Z = inf
{'mean': array(-0.06844031) * V, 'std': array(0.) * V} {'mean': array(-0.06522619) * V, 'std': arr